In [ ]:
# =============================================================== 
# Classificação Binária – Covertype (todas as 54 features)
# Spruce/Fir (1, classe positiva) vs Outras (0, classe negativa)
# Modelo: Regressão Logística (com StandardScaler)
# Saídas:
#   - visão do dataset + 10 primeiras linhas
#   - métricas (acc/prec/recall/F1) e matriz de confusão no TESTE
#   - predições individuais com probabilidades
#   - interpretação via coeficientes da RL
# ===============================================================

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, classification_report
)

RANDOM_STATE = 42
TEST_SIZE = 0.30

# ---------------------------------------------------------------
# 1) Carregar Covertype e tornar BINÁRIA: 1=Spruce/Fir, 0=Outras
# ---------------------------------------------------------------
cov = fetch_covtype(as_frame=True)
X = cov.data        # (581012, 54)
y_mult = cov.target # Rótulos de 1 a 7

# Binarizar: classe 1 (Spruce/Fir) -> 1 (positiva), outras -> 0 (negativa)
y = (y_mult == 1).astype(int)

feature_names = cov.feature_names
class_map = {0: "Outras (negativa)", 1: "Spruce/Fir (positiva)"}

print("=== Dataset: Covertype (binário) ===")
print("Classes: 0 = Outras (negativa), 1 = Spruce/Fir (positiva)")
print("Features:", list(feature_names))
print(f"Instâncias totais: {X.shape[0]} | Nº de features: {X.shape[1]}")

# Mostrar as 10 primeiras linhas (features)
df = pd.DataFrame(X)
df["classe"] = y.map(class_map)
print("\n=== 10 primeiras linhas ===")
# Usamos .to_string() para garantir que todas as colunas sejam mostradas
print(df.head(10).to_string())

# Distribuição global das classes
vals, cnts = np.unique(y, return_counts=True)
print("\nDistribuição por classe (total):")
for v, c in zip(vals, cnts):
    print(f"  classe {v} = {class_map[v]:<25s} -> {c}")

# ---------------------------------------------------------------
# 2) Split treino/teste (estratificado preserva proporções)
# ---------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)
print(f"\nTamanhos -> treino: {X_train.shape[0]} | teste: {X_test.shape[0]}")

# ---------------------------------------------------------------
# 3) Pipeline: padronização + Regressão Logística
# ---------------------------------------------------------------
# - A RL otimiza uma função que converge melhor quando as features têm escala similar.
# - Coeficientes ficam em escalas comparáveis (ajuda na interpretação relativa).
modelo = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))
])

# ---------------------------------------------------------------
# 4) Treinar
# ---------------------------------------------------------------
print("\nTreinando o modelo... (Isso pode levar alguns segundos)")
modelo.fit(X_train, y_train)
print("Treinamento concluído.")

# ---------------------------------------------------------------
# 5) Avaliar no TESTE
# ---------------------------------------------------------------
y_pred = modelo.predict(X_test)
cm = confusion_matrix(y_test, y_pred)  # linhas = Real, colunas = Previsto

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, pos_label=1)  # positiva = Spruce/Fir
rec  = recall_score(y_test, y_pred, pos_label=1)
f1   = f1_score(y_test, y_pred, pos_label=1)

print("\n=== Desempenho no TESTE ===")
print(f"Acurácia: {acc:.3f}")
print(f"Precisão (positiva=Spruce/Fir): {prec:.3f}")
print(f"Recall   (positiva=Spruce/Fir): {rec:.3f}")
print(f"F1       (positiva=Spruce/Fir): {f1:.3f}")

print("\nMatriz de Confusão (linhas=Real, colunas=Previsto):")
print("           Prev 0   Prev 1")
print(f"Real 0  |  {cm[0,0]:>6}   {cm[0,1]:>6}   <- Outras (0)")
print(f"Real 1  |  {cm[1,0]:>6}   {cm[1,1]:>6}   <- Spruce/Fir (1)")

print("\nRelatório por classe:")
print(classification_report(y_test, y_pred, target_names=["Outras", "Spruce/Fir"]))

# ---------------------------------------------------------------
# 6) Predições individuais (com 54 features)
# ---------------------------------------------------------------
# Pegamos 3 amostras do próprio conjunto de teste para exemplificar
amostras = X_test.head(3)

probas = modelo.predict_proba(amostras)  # colunas: [P(0), P(1)]
preds  = modelo.predict(amostras)

print("=== Predições Individuais ===")
for i in range(len(amostras)):
    x = amostras.iloc[i].values
    p = probas[i]
    pred = preds[i]
    print(f"\nAmostra (índice {amostras.index[i]}):")
    print(f"--> prev={'Spruce/Fir (1)' if pred==1 else 'Outras (0)'} "
          f"| P(Outras)= {p[0]:.3f} | P(Spruce/Fir)= {p[1]:.3f}")

# ---------------------------------------------------------------
# 7) Interpretação via Coeficientes da Regressão Logística
# ---------------------------------------------------------------
# Acessamos o estimador (clf) dentro do pipeline
coeficientes = modelo.named_steps['clf'].coef_.flatten()

# Criamos um DataFrame para visualizar a importância de cada feature
coef_df = pd.DataFrame({'Feature': feature_names, 'Coeficiente': coeficientes})
coef_df['Coef_Abs'] = np.abs(coef_df['Coeficiente'])
coef_df_sorted = coef_df.sort_values(by='Coef_Abs', ascending=False)

print("\n\n=== Interpretação do Modelo: Coeficientes da Regressão Logística ===")
print("Um coeficiente POSITIVO aumenta a chance da previsão ser 'Spruce/Fir (1)'.")
print("Um coeficiente NEGATIVO aumenta a chance da previsão ser 'Outras (0)'.")
print("\n--- 10 Features Mais Influentes ---")
print(coef_df_sorted.head(10).to_string(index=False))

=== Dataset: Covertype (binário) ===
Classes: 0 = Outras (negativa), 1 = Spruce/Fir (positiva)
Features: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Soil_Type_0', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3', 'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7', 'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11', 'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15', 'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19', 'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23', 'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27', 'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31', 'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35', 'Soil_Type_36', '